In [1]:
import os
import pandas as pd
from tools.dataset import score_dataset
from tools.preprocessing.missing_values import get_missing_raio, delete_columns, impute_missing_values
from tools.preprocessing.outliers import dcelete_outliers, impute_outliers, get_limits
from tools.preprocessing.scaling import minmax
from tools.engineering.mi import mi_score
from tools.engineering.encoding import one_hot
from tools.engineering.clustering import kmc


# 1. 데이터 로드

In [2]:
DATA_PATH = './data'

train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

# train_origin = train.copy()
# test_origin = test.copy()

print(f'train shape: {train.shape}')
print(f'test shape: {test.shape}')

train shape: (79023, 76)
test shape: (24353, 75)


In [3]:
# 데이터 일부 확인
train.head()

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,ID_-0.510_29.290_2019_00,-0.51,29.29,2019,0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,...,3664.436218,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140,3.750994
1,ID_-0.510_29.290_2019_01,-0.51,29.29,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,...,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779,4.025176
2,ID_-0.510_29.290_2019_02,-0.51,29.29,2019,2,0.000514,0.748199,0.000385,0.110018,72.795837,...,4216.986492,60068.894448,3516.282669,21.103410,0.251101,15.377883,30.401823,-142.519545,26.193296,4.231381
3,ID_-0.510_29.290_2019_03,-0.51,29.29,2019,3,NaN,NaN,NaN,NaN,NaN,...,5228.507736,51064.547339,4180.973322,15.386899,0.262043,-11.293399,24.380357,-132.665828,28.829155,4.305286
4,ID_-0.510_29.290_2019_04,-0.51,29.29,2019,4,-0.000079,0.676296,-0.000048,0.121164,4.121269,...,3980.598120,63751.125781,3355.710107,8.114694,0.235847,38.532263,37.392979,-141.509805,22.204612,4.347317


In [4]:
# train.drop(['ID_LAT_LON_YEAR_WEEK'], axis=1, inplace=True)
# test.drop(['ID_LAT_LON_YEAR_WEEK'], axis=1, inplace=True)

In [5]:
selected_columns = [
    'ID_LAT_LON_YEAR_WEEK',
    'latitude', 
    'longitude',
    'week_no',
    'year',
    'CarbonMonoxide_CO_column_number_density',
    'CarbonMonoxide_H2O_column_number_density',
    'SulphurDioxide_SO2_column_number_density',
    'NitrogenDioxide_NO2_column_number_density',
    'Formaldehyde_tropospheric_HCHO_column_number_density',
    'Ozone_O3_column_number_density',
    'emission',
]

In [6]:
train = train.loc[:, selected_columns]
selected_columns.remove('emission')
test = test.loc[:, selected_columns]

# 2. 결측값 처리

In [7]:
# 삭제하지 않을 칼럼
# protected_columns = ['latitude', 'longitude', 'week_no', 'year']

결측값 비율이 높은 칼럼 삭제

In [8]:
train_deleted, deleted_columns = delete_columns(train, 0.3, target='emission')

print(f'삭제된 칼럼 개수: {len(deleted_columns)}')

삭제된 칼럼 개수: 0


In [9]:
# 테스트 데이터에도 적용
test_deleted = test.drop(deleted_columns, axis=1)

In [10]:
# # 결측값 제거 결과
# data_list = [['train', train], ['train_deleted', train_deleted]]
# results = []

# for name, data in data_list:
#     score = score_dataset(data, 'emission')
#     results.append([name, score])

# results

칼럼을 삭제한 데이터셋의 스코어가 더 좋다.

가장 이상적인 결측값 대체법 찾기 

In [11]:
# methods = ['mean', 'linear', 'fill']
# results = []

# for method in methods:
#     train_imputed, _ = impute_missing_values(train_deleted, method)
#     score = score_dataset(train_imputed, 'emission')
#     results.append([method, score])
#     print(f'method "{method}" 계산 완료')

# results

In [12]:
# best_method = sorted(results, key=lambda x: x[1])[0]
# print(f'best method: {best_method}')

In [13]:
# train_imputed, _ = impute_missing_values(train_deleted, best_method[0])
# test_imputed, _ = impute_missing_values(test_deleted, best_method[0])

train_imputed, _ = impute_missing_values(train_deleted, 'fill')
test_imputed, _ = impute_missing_values(test_deleted, 'fill')

In [14]:
train_now = train_imputed
test_now = test_imputed

In [15]:
# score_dataset(train_now, 'emission')

# 3. 칼럼 추가

month 및 covid 칼럼 추가

In [16]:
train_now['date'] = pd.to_datetime('2021' + train_now['week_no'].astype(str) + '0', format='%Y%W%w')
train_now['month_no'] = train_now['date'].dt.month
train_now.drop(columns=['date'], inplace=True)

train_now['covid'] = (train_now.year == 2020) & (train_now.month_no > 2)

In [17]:
test_now['date'] = pd.to_datetime('2021' + test_now['week_no'].astype(str) + '0', format='%Y%W%w')
test_now['month_no'] = test_now['date'].dt.month
test_now.drop(columns=['date'], inplace=True)

test_now['covid'] = (test_now.year == 2020) & (test_now.month_no > 2)

In [18]:
train_now, encoder = one_hot(train_now, 'covid')
test_now, _ = one_hot(test_now, 'covid', encoder)

c:\Users\loven\dev\kaggle_competition\carbon\carbon\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [19]:
train_now.rename(columns={0: 'covid_false', 1: 'covid_true'}, inplace=True)
test_now.rename(columns={0: 'covid_false', 1: 'covid_true'}, inplace=True)

In [20]:
# protected_columns.extend(['covid_false', 'covid_true', 'month_no'])

season 칼럼 추가

In [21]:
SEASON = {
    'summer': [12, 1, 2],
    'fall': [3, 4, 5],
    'winter': [6, 7, 8],
    'spring': [9, 10, 11],
}

In [22]:
def insert_season(x, season):
    if x in season['summer']:
        return 'summer'
    elif x in season['fall']:
        return 'fall'
    elif x in season['winter']:
        return 'winter'
    elif x in season['spring']:
        return 'spring'
    else:
        raise Exception('unknown week')

In [23]:
train_now['season'] = train_now['month_no'].apply(insert_season, args=[SEASON])

test_now['season'] = test_now['month_no'].apply(insert_season, args=[SEASON])

print(train_now['season'].value_counts(), test_now['season'].value_counts())

season
summer    20874
fall      19383
winter    19383
spring    19383
Name: count, dtype: int64 season
fall      6461
winter    6461
spring    6461
summer    4970
Name: count, dtype: int64


계절별 데이터셋 따로 구성

In [24]:
drop_columns = ['week_no', 'month_no']
train_now = train_now.drop(drop_columns, axis=1)
test_now = test_now.drop(drop_columns, axis=1)

In [25]:
from sklearn.manifold import TSNE
tsne_columns = [    
    'CarbonMonoxide_CO_column_number_density',
    'CarbonMonoxide_H2O_column_number_density',
    'SulphurDioxide_SO2_column_number_density',
    'NitrogenDioxide_NO2_column_number_density',
    'Formaldehyde_tropospheric_HCHO_column_number_density',
    'Ozone_O3_column_number_density',
]

tsne = TSNE()

In [26]:
tsne_train = train_now.loc[:, tsne_columns]
tsne_test = test_now.loc[:, tsne_columns]

In [27]:
tsne_train, scaler = minmax(tsne_train, tsne_columns)
tsne_test, _ = minmax(tsne_test, tsne_columns, scaler=scaler)

In [29]:
tsne_train_tf = pd.DataFrame(tsne.fit_transform(tsne_train), columns = ['component 0', 'component 1'])
tsne_test_tf = pd.DataFrame(tsne.fit_transform(tsne_test), columns = ['component 0', 'component 1'])

In [31]:
train_now = train_now.drop(tsne_columns, axis=1)
test_now = test_now.drop(tsne_columns, axis=1)

train_now = train_now.reset_index(drop=True)
test_now = test_now.reset_index(drop=True)

In [32]:
train_now = pd.concat([train_now, tsne_train_tf], axis=1)
test_now = pd.concat([test_now, tsne_test_tf], axis=1)

In [33]:
train_now

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,emission,covid_False,covid_True,season,component 0,component 1
0,ID_-0.510_29.290_2019_00,-0.510,29.290,2019,3.750994,1.0,0.0,summer,41.872795,28.055298
1,ID_-0.510_29.290_2019_01,-0.510,29.290,2019,4.025176,1.0,0.0,summer,11.623838,47.083565
2,ID_-0.510_29.290_2019_02,-0.510,29.290,2019,4.231381,1.0,0.0,summer,29.817415,60.247566
3,ID_-0.510_29.290_2019_03,-0.510,29.290,2019,4.305286,1.0,0.0,summer,35.033493,60.075359
4,ID_-0.510_29.290_2019_04,-0.510,29.290,2019,4.347317,1.0,0.0,summer,2.674914,56.503468
...,...,...,...,...,...,...,...,...,...,...
79018,ID_-3.299_30.301_2021_48,-3.299,30.301,2021,29.404171,1.0,0.0,summer,63.680889,-31.599283
79019,ID_-3.299_30.301_2021_49,-3.299,30.301,2021,29.186497,1.0,0.0,summer,65.941086,-35.343098
79020,ID_-3.299_30.301_2021_50,-3.299,30.301,2021,29.131205,1.0,0.0,summer,63.226662,-24.261408
79021,ID_-3.299_30.301_2021_51,-3.299,30.301,2021,28.125792,1.0,0.0,summer,88.782661,-16.516949


In [34]:
train_summer = train_now.loc[train_now['season'] == 'summer']
train_fall = train_now.loc[train_now['season'] == 'fall']
train_winter = train_now.loc[train_now['season'] == 'winter']
train_spring = train_now.loc[train_now['season'] == 'spring']

test_summer = test_now.loc[test_now['season'] == 'summer']
test_fall = test_now.loc[test_now['season'] == 'fall']
test_winter = test_now.loc[test_now['season'] == 'winter']
test_spring = test_now.loc[test_now['season'] == 'spring']

In [35]:
train_sets = [train_summer, train_fall, train_winter, train_spring]
test_sets = [test_summer, test_fall, test_winter, test_spring]

In [36]:
for i in range(len(train_sets)):
    train_sets[i] = train_sets[i].drop('season', axis=1)

for i in range(len(test_sets)):
    test_sets[i] = test_sets[i].drop('season', axis=1)

# 4. Feature Selection(by Mutual Information, Correlation)

In [24]:
selected_columns = []

for idx, train in enumerate(train_sets):
    df, columns = mi_score(train, 'emission', 0.3, corr=True, corr_threshold=0.3, protected=protected_columns)
    columns.remove('emission')
    selected_columns.append(columns)
    train_sets[idx] = df
    print(f'{idx} train의 칼럼 개수: {len(train_sets[idx].columns)}')

0 train의 칼럼 개수: 20
1 train의 칼럼 개수: 8
2 train의 칼럼 개수: 8
3 train의 칼럼 개수: 9


In [37]:
score_dataset(train_sets[0], 'emission')

8.588228397127487

In [41]:
score_dataset(temp, 'emission')

8.562238400578966

In [25]:
for idx, test in enumerate(test_sets):
    test_sets[idx] = test.loc[:, selected_columns[idx]]

# 5. Clustering

In [26]:
# features = ['latitude', 'longitude']

In [27]:
# clusters = []
# encoders = []
# for idx, train in enumerate(train_sets):
#     df, cluster, encoder, n_clusters = kmc(train, features, n_clusters=5, elbow=True, encoding=True)
#     encoders.append(encoder)
#     train_sets[idx] = df
#     clusters.append(cluster)

The elbow method is excecuting


c:\Users\loven\dev\kaggle_competition\carbon\carbon\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


The elbow method is excecuting


c:\Users\loven\dev\kaggle_competition\carbon\carbon\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


The elbow method is excecuting


c:\Users\loven\dev\kaggle_competition\carbon\carbon\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


The elbow method is excecuting


c:\Users\loven\dev\kaggle_competition\carbon\carbon\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [28]:
# for idx, test in enumerate(test_sets):
#     df, _, _, _ = kmc(test, features, cluster=clusters[idx], encoding=True, encoder=encoders[idx])
#     test_sets[idx] = df

# 7. 최종 데이터셋

In [37]:
season_order = list(SEASON.keys())
season_order

['summer', 'fall', 'winter', 'spring']

In [38]:
train_columns = []

for idx, train in enumerate(train_sets):
    col = list(map(str, list(train.columns)))
    train.columns = col
    col.sort()
    train_reordered = train[col]
    col.remove('emission')
    train_columns.append(col)
    data_name = os.path.join(DATA_PATH, '0809/train_{}_2.csv'.format(season_order[idx]))
    train_reordered.to_csv(data_name)

In [39]:
for idx, test in enumerate(test_sets):
    col = list(map(str, list(test.columns)))
    test.columns = col
    test_reordered = test[train_columns[idx]]
    data_name = os.path.join(DATA_PATH, '0809/test_{}_2.csv'.format(season_order[idx]))
    test_reordered.to_csv(data_name)